In [2]:
#Imports
from math import exp
from random import seed
from random import random

# Methods

In [15]:
## Initialize the Network
def initialize(nInputs,nHidden,nOutputs):
    nn = list()
    hidden_layer = [{'weights':[random() for i in range(nInputs + 1)]} for i in range(nHidden)]
    nn.append(hidden_layer)
    output_layer = [{'weights':[random() for i in range(nHidden + 1)]} for i in range(nOutputs)]
    nn.append(output_layer)
    return nn
##EX: Inputs Based on HW example
nnTest = initialize(2,3,2)
nnTest

[[{'weights': [0.04867004643405559, 0.34191875689337337, 0.7675190728202232]},
  {'weights': [0.9041471736555495, 0.8461087955749224, 0.6973124996479755]},
  {'weights': [0.33395117659733675, 0.6364528636580962, 0.21585368592083076]}],
 [{'weights': [0.9478778318343694,
    0.48620389024745814,
    0.4138452177977743,
    0.423569531962366]},
  {'weights': [0.5674600098512985,
    0.3647822041420016,
    0.25464210235046614,
    0.5377591766164905]}]]

### Forward Propogation

In [16]:
def netInput(weights,inputs):
    netInput = weights[-1]
    for i in range(len(weights)-1):
        netInput = netInput + weights[i]*inputs[i]
    return netInput
def netOutput(netInput):
    return 1.0 / (1.0 + exp(-netInput))
def forwardPropagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = netInput(neuron['weights'], inputs)
            neuron['output'] = netOutput(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs
## Testing Example 
row = [.6, .1, None]
output = forwardPropagate(nnTest, row)
print(output)
print('-----------')
print(nnTest)

[0.8485804944130987, 0.7987445048958676]
-----------
[[{'weights': [0.04867004643405559, 0.34191875689337337, 0.7675190728202232], 'output': 0.6965479392627051}, {'weights': [0.9041471736555495, 0.8461087955749224, 0.6973124996479755], 'output': 0.7899147592724305}, {'weights': [0.33395117659733675, 0.6364528636580962, 0.21585368592083076], 'output': 0.6177171007123751}], [{'weights': [0.9478778318343694, 0.48620389024745814, 0.4138452177977743, 0.423569531962366], 'output': 0.8485804944130987}, {'weights': [0.5674600098512985, 0.3647822041420016, 0.25464210235046614, 0.5377591766164905], 'output': 0.7987445048958676}]]


### Back Propogation

In [18]:
def deriv(output):
    return output*(1.0 - output)
def backwardPropagate(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * deriv(neuron['output'])

# Update network weights with error
def newWeights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']
backwardPropagate(nnTest,[1,0])
nnTest

[[{'weights': [0.04867004643405559, 0.34191875689337337, 0.7675190728202232],
   'output': 0.6965479392627051,
   'delta': -0.011502604587742963},
  {'weights': [0.9041471736555495, 0.8461087955749224, 0.6973124996479755],
   'output': 0.7899147592724305,
   'delta': -0.006202894560279114},
  {'weights': [0.33395117659733675, 0.6364528636580962, 0.21585368592083076],
   'output': 0.6177171007123751,
   'delta': -0.005819524302127215}],
 [{'weights': [0.9478778318343694,
    0.48620389024745814,
    0.4138452177977743,
    0.423569531962366],
   'output': 0.8485804944130987,
   'delta': 0.019456140436517503},
  {'weights': [0.5674600098512985,
    0.3647822041420016,
    0.25464210235046614,
    0.5377591766164905],
   'output': 0.7987445048958676,
   'delta': -0.12839955363717998}]]

### Train

In [29]:
def train_network(network, data, lrate, n_iteration, n_outputs):
    for iteration in range(n_iteration):
        sum_error = 0
        for row in data:
            outputs= forwardPropagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backwardPropagate(nn, expected)
            newWeights(nn, row, lrate)
    return nn

## Test Example $\Rightarrow$ From HW

In [30]:
nn = initialize(2,3,2)
trainingSet = [[.6,.1,1],[.2,.3,0]] # 1 for screw, 0 for nail
## 20 iterations because why not
train_network(nn,trainingSet,.1, 1000, 2 )

[[{'weights': [0.6984730471163791, 0.06836983190070531, 0.42486325208294184],
   'output': 0.6428846723050125,
   'delta': -0.025112937339829354},
  {'weights': [0.32133912559117844, 1.1234246476745002, 0.6911741021378273],
   'output': 0.7485283651877953,
   'delta': 0.01495948235914266},
  {'weights': [0.6198280305139938, -0.06370547311781251, -0.08996253202182175],
   'output': 0.5044124575484605,
   'delta': -0.02441465711568682}],
 [{'weights': [-0.10716265359179475,
    0.33871451893595894,
    -0.3220117245503351,
    -0.017367527523875513],
   'output': 0.49416993804819,
   'delta': 0.12644032251516743},
  {'weights': [0.7505018434528816,
    -0.3121273699924513,
    0.44289962294338814,
    -0.5003135879626456],
   'output': 0.4999469569835379,
   'delta': -0.1249867378392529}]]

 Use the nueral network algorithm to run prediction models after MANY iterations (1000)